In [1]:
import torch

# 檢查系統中是否有可用的 CUDA
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available. Using CPU instead.")

CUDA is available!


In [2]:
# %load train.py
from argparse import ArgumentParser
from typing import Tuple


import numpy as np
import torch
import torch.nn.functional as F
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.types import STEP_OUTPUT, EPOCH_OUTPUT
from pytorch_lightning.callbacks import ModelCheckpoint

from dataset.mpii_face_gaze_dataset import get_dataloaders
from model import FinalModel
from utils import calc_angle_error, PitchYaw, plot_prediction_vs_ground_truth, log_figure, get_random_idx, get_each_of_one_grid_idx
import os





INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [3]:
class Model(FinalModel):
    def __init__(self, learning_rate: float = 0.001, weight_decay: float = 0., k=None, adjust_slope: bool = False, grid_calibration_samples: bool = False, *args, **kwargs):
        print("Initializing train Model...")
        super().__init__(*args, **kwargs)

        print("train Model base (FinalModel) initialized.")
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.k = [9, 128] if k is None else k
        self.adjust_slope = adjust_slope
        self.grid_calibration_samples = grid_calibration_samples

        self.save_hyperparameters()  # log hyperparameters

        print("train Model initialization completed.")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

    def __step(self, batch: dict) -> Tuple:
        """
        Operates on a single batch of data.

        :param batch: The output of your :class:`~torch.utils.data.DataLoader`. A tensor, tuple or list.
        :return: calculated loss, given values and predicted outputs
        """
        person_idx = batch['person_idx'].long()
        left_eye_image = batch['left_eye_image'].float()
        right_eye_image = batch['right_eye_image'].float()
        full_face_image = batch['full_face_image'].float()

        gaze_pitch = batch['gaze_pitch'].float()
        gaze_yaw = batch['gaze_yaw'].float()

        #print("gaze_pitch:", gaze_pitch)
        #print("gaze_yaw:", gaze_yaw)
        
        labels = torch.stack([gaze_pitch, gaze_yaw]).T

        outputs = self(person_idx, full_face_image, right_eye_image, left_eye_image)  # prediction on the base model
        loss = F.mse_loss(outputs, labels)

        return loss, labels, outputs

    def training_step(self, train_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(train_batch)

        self.log('train/loss', loss)
        self.log('train/angular_error', calc_angle_error(labels, outputs))

        return loss

    def validation_step(self, valid_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(valid_batch)

        self.log('valid/offset(k=0)/loss', loss)
        self.log('valid/offset(k=0)/angular_error', calc_angle_error(labels, outputs))


        return {'loss': loss, 'labels': labels, 'outputs': outputs, 'gaze_locations': valid_batch['gaze_location'], 'screen_sizes': valid_batch['screen_size']}

    def validation_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        self.__log_and_plot_details(outputs, 'epoch_end_valid')

    def test_step(self, test_batch: dict, batch_idx: int) -> STEP_OUTPUT:
        loss, labels, outputs = self.__step(test_batch)

        self.log('test/offset(k=0)/loss', loss,on_epoch=True, prog_bar=True)
        self.log('test/offset(k=0)/angular_error', calc_angle_error(labels, outputs))

        return {'loss': loss, 'labels': labels, 'outputs': outputs, 'gaze_locations': test_batch['gaze_location'], 'screen_sizes': test_batch['screen_size']}

    def test_epoch_end(self, outputs: EPOCH_OUTPUT) -> None:
        self.__log_and_plot_details(outputs, 'epoch_end_test')

    def __log_and_plot_details(self, outputs, tag: str):
        test_labels = torch.cat([output['labels'] for output in outputs])
        test_outputs = torch.cat([output['outputs'] for output in outputs])
        test_gaze_locations = torch.cat([output['gaze_locations'] for output in outputs])
        test_screen_sizes = torch.cat([output['screen_sizes'] for output in outputs])

        figure = plot_prediction_vs_ground_truth(test_labels, test_outputs, PitchYaw.PITCH)
        log_figure(self.logger, f'{tag}/offset(k=0)/pitch', figure, self.global_step)

        figure = plot_prediction_vs_ground_truth(test_labels, test_outputs, PitchYaw.YAW)
        log_figure(self.logger, f'{tag}/offset(k=0)/yaw', figure, self.global_step)

        # find calibration params
         # 確保 last_x 合理
        last_x = min(500, len(test_labels))
        calibration_train = test_outputs[:-last_x].cpu().detach().numpy()
        calibration_test = test_outputs[-last_x:].cpu().detach().numpy()
        calibration_train_labels = test_labels[:-last_x].cpu().detach().numpy()
        calibration_test_labels = test_labels[-last_x:].cpu().detach().numpy()

        # 日誌檢查
        print(f"日誌檢查開始")
        print(f"test_labels shape: {test_labels.shape}")
        print(f"last_x: {last_x}")
        print(f"calibration_train_labels shape: {calibration_train_labels.shape}")
        print(f"calibration_train shape: {calibration_train.shape}")

        


        # 檢查切片後的結果是否為空##############################################################

        # 檢查 calibration_train_labels 是否為空
        if calibration_train_labels.size == 0:
            print("Warning: calibration_train_labels is empty! Skipping this batch.")
            return


        ######################################################################################
        gaze_locations_train = test_gaze_locations[:-last_x].cpu().detach().numpy()
        screen_sizes_train = test_screen_sizes[:-last_x].cpu().detach().numpy()

        if len(calibration_train) > 0:
            for k in self.k:
                if k <= 0:
                    continue
                calibrated_solutions = []

                num_calibration_runs = 500 if self.grid_calibration_samples else 10_000  # original results are both evaluated with 10,000 runs
                for calibration_run_idx in range(num_calibration_runs):  # get_each_of_one_grid_idx is slower than get_random_idx
                    np.random.seed(42 + calibration_run_idx)
                    calibration_sample_idxs = get_each_of_one_grid_idx(k, gaze_locations_train, screen_sizes_train) if self.grid_calibration_samples else get_random_idx(k, len(calibration_train))

                    #####################################################################
                    ###加判斷
                    #######################################################################
                    # 確保索引不會超出範圍
                    if not calibration_sample_idxs:
                        print(f"Warning: No valid calibration sample indices for k={k}. Skipping this iteration.")
                        continue
               
                    #######################################################################

                    
                    calibration_points_x = np.asarray([calibration_train[idx] for idx in calibration_sample_idxs])
                    calibration_points_y = np.asarray([calibration_train_labels[idx] for idx in calibration_sample_idxs])

                     #####################################################################
                    ###加判斷
                    #######################################################################

                    # 檢查 calibration_points 是否為空
                    if calibration_points_x.size == 0 or calibration_points_y.size == 0:
                        print(f"Warning: Empty calibration points for k={k}. Skipping this iteration.")
                        continue

                     #####################################################################
            

                    if self.adjust_slope:
                        m, b = np.polyfit(calibration_points_y[:, :1].reshape(-1), calibration_points_x[:, :1].reshape(-1), deg=1)
                        pitch_fixed = (calibration_test[:, :1] - b) * (1 / m)
                        m, b = np.polyfit(calibration_points_y[:, 1:].reshape(-1), calibration_points_x[:, 1:].reshape(-1), deg=1)
                        yaw_fixed = (calibration_test[:, 1:] - b) * (1 / m)
                    else:
                        mean_diff_pitch = (calibration_points_y[:, :1] - calibration_points_x[:, :1]).mean()  # mean offset
                        pitch_fixed = calibration_test[:, :1] + mean_diff_pitch
                        mean_diff_yaw = (calibration_points_y[:, 1:] - calibration_points_x[:, 1:]).mean()  # mean offset
                        yaw_fixed = calibration_test[:, 1:] + mean_diff_yaw

                    pitch_fixed, yaw_fixed = torch.Tensor(pitch_fixed), torch.Tensor(yaw_fixed)
                    outputs_fixed = torch.stack([pitch_fixed, yaw_fixed], dim=1).squeeze(-1)
                    calibrated_solutions.append(calc_angle_error(torch.Tensor(calibration_test_labels), outputs_fixed).item())

            #####################################################################
            ###加判斷
            #######################################################################
                if calibrated_solutions:
                    self.log(f'{tag}/offset(k={k})/mean_angular_error', np.asarray(calibrated_solutions).mean())
                    self.log(f'{tag}/offset(k={k})/std_angular_error', np.asarray(calibrated_solutions).std())
                else:
                    print(f"Warning: No calibrated solutions generated for k={k}.")

         #######################################################################

        # best case, with all calibration samples, all values except the last `last_x` values
        if self.adjust_slope:
            m, b = np.polyfit(calibration_train_labels[:, :1].reshape(-1), calibration_train[:, :1].reshape(-1), deg=1)
            pitch_fixed = torch.Tensor((calibration_test[:, :1] - b) * (1 / m))
            m, b = np.polyfit(calibration_train_labels[:, 1:].reshape(-1), calibration_train[:, 1:].reshape(-1), deg=1)
            yaw_fixed = torch.Tensor((calibration_test[:, 1:] - b) * (1 / m))
        else:
            mean_diff_pitch = (calibration_train_labels[:, :1] - calibration_train[:, :1]).mean()  # mean offset
            pitch_fixed = calibration_test[:, :1] + mean_diff_pitch
            mean_diff_yaw = (calibration_train_labels[:, 1:] - calibration_train[:, 1:]).mean()  # mean offset
            yaw_fixed = calibration_test[:, 1:] + mean_diff_yaw

        pitch_fixed, yaw_fixed = torch.Tensor(pitch_fixed), torch.Tensor(yaw_fixed)
        outputs_fixed = torch.stack([pitch_fixed, yaw_fixed], dim=1).squeeze(-1)
        calibration_test_labels = torch.Tensor(calibration_test_labels)
        self.log(f'{tag}/offset(k=all)/angular_error', calc_angle_error(calibration_test_labels, outputs_fixed))

        figure = plot_prediction_vs_ground_truth(calibration_test_labels, outputs_fixed, PitchYaw.PITCH)
        log_figure(self.logger, f'{tag}/offset(k=all)/pitch', figure, self.global_step)

        figure = plot_prediction_vs_ground_truth(calibration_test_labels, outputs_fixed, PitchYaw.YAW)
        log_figure(self.logger, f'{tag}/offset(k=all)/yaw', figure, self.global_step)

    




In [4]:
def main(path_to_data: str, validate_on_person: int, test_on_person: int, learning_rate: float, weight_decay: float, batch_size: int, k: int, adjust_slope: bool, grid_calibration_samples: bool):
    seed_everything(42)

    model = Model(learning_rate, weight_decay, k, adjust_slope, grid_calibration_samples)

    # 確保保存模型的目錄存在(變成eval要的型態資料夾)
    base_model_dir = './saved_models/'
    os.makedirs(base_model_dir, exist_ok=True)
    person_model_dir = os.path.join(base_model_dir, f'p{validate_on_person:02d}')
    os.makedirs(person_model_dir, exist_ok=True)


    # 設定模型保存
    checkpoint_callback = ModelCheckpoint(
        dirpath=person_model_dir,  # 保存模型的目錄
        filename=f'p{validate_on_person:02d}_best',  # 保存文件的名稱
        save_top_k=1,  # 只保存最好的模型
        verbose=True,
        monitor='valid/offset(k=0)/loss',  # 根據某個監控指標保存，比如 validation loss
        mode='min'  # 以最小化 val_loss 的方式保存最好的模型
    )

    trainer = Trainer(
        gpus=1,
        max_epochs=1,
        fast_dev_run=False,
        default_root_dir=person_model_dir,
        logger=[
            TensorBoardLogger(save_dir=f"tb_logs/p{validate_on_person:02d}"),
        ],
        benchmark=True,
        callbacks=[checkpoint_callback],  # 傳入模型保存回調

        ######禁用檢查要開才能印出model的print
        #num_sanity_val_steps=0,
        #limit_val_batches=0,  # 禁用验证数据检查
        #enable_progress_bar=True,  # 确保显示标准输出
        #log_every_n_steps=1
    )

    print("Start data loading...")
    train_dataloader, valid_dataloader, test_dataloader = get_dataloaders(path_to_data, validate_on_person, test_on_person, batch_size)

    print("Data loading complete.")
  
    trainer.fit(model, train_dataloader, valid_dataloader)
    trainer.test(model, test_dataloader)




In [5]:
import argparse
import sys

path_to_data = './data/mpiifacegaze_preprocessed'
validate_on_person = 0
test_on_person = 1
learning_rate = 0.001
weight_decay = 0
batch_size = 32
k = [9, 128]
adjust_slope = False
grid_calibration_samples = True

main(path_to_data, validate_on_person, test_on_person, learning_rate, weight_decay, batch_size, k, adjust_slope, grid_calibration_samples)

Global seed set to 42


Initializing train Model...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train Model base (FinalModel) initialized.
train Model initialization completed.
Start data loading...
train on persons [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
valid on person 0
test on person 1
len(dataset_train) 60784
len(dataset_valid) 2927


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | cnn_face     | Sequential | 683 K 
1 | cnn_eye      | Sequential | 564 K 
2 | fc_face      | Sequential | 1.2 M 
3 | cnn_eye2fc   | Sequential | 904 K 
4 | fc_eye       | Sequential | 1.6 M 
5 | fc_eyes_face | Sequential | 148 K 
--------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.289    Total estimated model params size (MB)


len(dataset_test) 2904
Data loading complete.
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.27s/it]日誌檢查開始
test_labels shape: torch.Size([64, 2])
last_x: 64
calibration_train_labels shape: (0, 2)
calibration_train shape: (0, 2)
Epoch 0:  95%|█████████▌| 1899/1990 [03:08<00:09, 10.06it/s, loss=0.0226, v_num=29]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 1990/1990 [03:21<00:00,  9.87it/s, loss=0.0226, v_num=29]日誌檢查開始
test_labels shape: torch.Size([2912, 2])
last_x: 500
calibration_train_labels shape: (2412, 2)
calibration_train shape: (2412, 2)
Epoch 0: 100%|██████████| 1990/1990 [03:32<00:00,  9.36it/s, loss=0.0226, v_num=29]

Epoch 0, global step 1899: 'valid/offset(k=0)/loss' reached 0.04284 (best 0.04284), saving model to './saved_models/p00\\p00_best.ckpt' as top 1


Epoch 0: 100%|██████████| 1990/1990 [03:32<00:00,  9.36it/s, loss=0.0226, v_num=29]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 90/90 [00:07<00:00, 11.14it/s]日誌檢查開始
test_labels shape: torch.Size([2880, 2])
last_x: 500
calibration_train_labels shape: (2380, 2)
calibration_train shape: (2380, 2)
Testing DataLoader 0: 100%|██████████| 90/90 [00:17<00:00,  5.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                  Test metric                                   DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
epoch_end_test/offset(k=128)/mean_angular_error              11.068146989822388
epoch_end_test/offset(k=128)/std_angular_error              0.012213014576228998
 epoch_end_test/offset(k=9)/mean_angular_error               11.123067573547363
 epoch_end_test/offset(k=9)/std_angular_error                0.07988418199430629
  epoch_end_test/offset(k=all)/angular_error                 11.072676658630371
   